In [44]:
# imports
import sys
import pandas as pd
import numpy as np
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import Data.data_provider as dp
import core as cal
from estimators.IR_RF_estimator import IR_RF
from sklearn.ensemble import RandomForestClassifier


In [45]:
# params
calib_methods = cal.calib_methods.copy()
metrics = ["acc", "brier"]

data_list = ["spambase", "climate", "QSAR", "bank", "climate", "parkinsons", "vertebral", "ionosphere", "diabetes", "breast", "blod"]
# data_list = ["spambase", "climate"]

params = {
    "runs": 1,
    "n_estimators": 10,
    "oob": False,
    "test_split": 0.3,
    "calib_split": 0.01
}

In [46]:
calib_results_dict = {}

for data_name in data_list:

    # Data
    X, y = dp.load_data(data_name, "../../")
    
    data_dict = {} # results for each data set will be saved in here.
    for seed in range(params["runs"]): # running the same dataset multiple times
        # split the data
        data = cal.split_train_calib_test(data_name, X, y, params["test_split"], params["calib_split"], seed)
        models = {}
        # train model
        rf = RandomForestClassifier(n_estimators=params["n_estimators"], oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF_pet"] = rf

        irrf_l = IR_RF(n_estimators=params["n_estimators"], oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF_l"] = irrf_l

        irrf_ct = IR_RF(n_estimators=params["n_estimators"], oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF_ct"] = irrf_ct

        # calibration
        res = cal.model_calibration(models, data, metrics) # res is a dict with all the metrics results as well as RF probs and every calibration method decision for every test data point
        data_dict = cal.update_runs(data_dict, res) # calib results for every run for the same dataset is aggregated in data_dict (ex. acc of every run as an array)
    calib_results_dict.update(data_dict) # merge results of all datasets together

In [47]:
tables = cal.mean_and_ranking_table(calib_results_dict, metrics, models, data_list, mean_and_rank=True)

In [48]:
tables["brier"]

,RF_pet,RF_l,RF_ct
Data,,,
spambase,0.049509,0.050789,0.049500
climate,0.102222,0.100624,0.102222
QSAR,0.098549,0.099237,0.098549
bank,0.005097,0.006898,0.005097
climate,0.102222,0.100624,0.102222
parkinsons,0.073729,0.079078,0.073729
vertebral,0.107957,0.107830,0.107957
ionosphere,0.076792,0.078873,0.076792
diabetes,0.181775,0.171900,0.181775


In [49]:
tables["acc"]

,RF_pet,RF_l,RF_ct
Data,,,
spambase,0.932657,0.934106,0.932657
climate,0.858025,0.870370,0.858025
QSAR,0.870662,0.867508,0.870662
bank,0.990291,0.992718,0.990291
climate,0.858025,0.870370,0.858025
parkinsons,0.915254,0.881356,0.915254
vertebral,0.849462,0.827957,0.849462
ionosphere,0.905660,0.905660,0.905660
diabetes,0.727273,0.740260,0.727273
